<a href="https://colab.research.google.com/github/yanFORray/py_data/blob/main/carsS3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import csv
from collections import defaultdict
from pyhive import hive

In [47]:
# Загрузка данных из CSV-файла
def load_data(file_path):
    with open(file_path, "r") as file:
        reader = csv.DictReader(file)
        data = [row for row in reader]
    return data

In [48]:
# Вычисление средней цены для каждой марки
def calculate_avg_prices(data):
    prices_by_brand = defaultdict(list)
    for row in data:
        brand = row["Make"]
        price = float(row["MSRP"])
        prices_by_brand[brand].append(price)

    avg_prices_by_brand = {brand: sum(prices) / len(prices) for brand, prices in prices_by_brand.items()}
    return avg_prices_by_brand

In [49]:
# Сохранение результатов в новый CSV-файл
def save_to_csv(data, output_filename):
    with open(output_filename, "w", newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Brand", "Avg Price"])
        for brand, avg_price in data.items():
            writer.writerow([brand, avg_price])

if __name__ == "__main__":
    file_path = '/content/datacar.csv'
    data = load_data(file_path)

    avg_prices_by_brand = calculate_avg_prices(data)

    output_filename = "/content/avg_prices_by_brand.csv"
    save_to_csv(avg_prices_by_brand, output_filename)


In [ ]:

# Функция для загрузки данных в Hive
def load_into_hive(csv_file, table_name, host_name, port):
    conn = hive.Connection(host=host_name, port=port)

    cursor = conn.cursor()
    cursor.execute(f"CREATE TABLE IF NOT EXISTS {table_name} (brand STRING, avg_price FLOAT)")
    cursor.execute(f"LOAD DATA LOCAL INPATH '{csv_file}' OVERWRITE INTO TABLE {table_name}")

# Функция для анализа данных в Hive
def analyze_data_in_hive(table_name, host_name, port):
    conn = hive.Connection(host=host_name, port=port)

    cursor = conn.cursor()
    cursor.execute(f"SELECT brand, MIN(avg_price) AS min_price, MAX(avg_price) AS max_price FROM {table_name} GROUP BY brand")

    result = cursor.fetchall()
    for row in result:
        print(row)

if __name__ == "__main__":
    file_path = '/content/datacar.csv'
    data = load_data(file_path)

    avg_prices_by_brand = calculate_avg_prices(data)

    output_filename = "/content/avg_prices_by_brand.csv"
    save_to_csv(avg_prices_by_brand, output_filename)  # Сохраняем в файл CSV

    table_name = 'car_prices'
    host_name = 'your_hive_host'
    port = 10000

    load_into_hive(output_filename, table_name, host_name, port)  # Загружаем CSV-файл в Hive
    analyze_data_in_hive(table_name, host_name, port)  # Анализируем данные в Hive